In [ ]:
import pandas as pd

main_cities = [line.strip() for line in open("DATASET/main_cities")]

In [ ]:
import re

def is_new_journey(string):
    regex_header = r"-- ([0-9]+) --"
    matches_it = re.finditer(regex_header, line, re.MULTILINE)
    matches = [m for m in matches_it]
    return len(matches) == 1

regex_line = r"^([0-9]{7}) (.{20}) .([0-9\ ]{5}) .([0-9\ ]{5})"
def is_line_timetable(string):
    matches_it = re.finditer(regex_line, line, re.MULTILINE)
    matches = [m for m in matches_it]
    return len(matches) == 1

def get_line_timetable(line):
    if is_line_timetable(line):
        matches_it = re.finditer(regex_line, line, re.MULTILINE)
        matches = [m for m in matches_it]
        match = matches[0]
        return match.groups()

    return None, None, None, None

def is_start_journey(string):
    return string.strip() == ""


cities = set()
links = []
count_cities = {}
old = None
with open("DATASET/FPLAN", "r") as f:
    for line in f:
        if is_new_journey(line):
            #print("\n New Journey \n")
            old=None
            pass
        
        if is_line_timetable(line):
            number, name, start_str, end_str = get_line_timetable(line)
            name = name.strip()
            if name == "Haute-Nendaz, téléca":
                name = "Haute-Nendaz, télécabine"
            if name == "Verbier, station pos":
                name = "Verbier, station poste"
            
            count_cities[name.split(",")[0]] = count_cities.get(name.split(",")[0],0) + 1
            if name in main_cities:             
                cities.add(name)
                if old != None:
                    if old < name:
                        links.append((old, name))
                    else:
                        links.append((name, old))
            

                old = name
                          
print(cities)
print(links)

In [ ]:
from collections import Counter
count_dict = dict(Counter(links))
count_dict

In [ ]:
from datetime import date

regex_geo = r"^[0-9]{7} +([0-9\.\-]+)  +([0-9\.\-]+) .+\% (.+)"
def get_groups(string, regex): 
    matches_it = re.finditer(regex, string, re.MULTILINE)
    matches = [m for m in matches_it]
    match = matches[0]
    return match.groups()

cities_to_point = pd.DataFrame([], columns=["name", "long","lat", "date", "radius"])
with open("DATASET/BFKOORD_GEO", "r") as f:
    for line in f: 
        y,x,name = get_groups(line, regex_geo)
        if name in main_cities:
            cities_to_point = cities_to_point.append({"name" : name, "lat" : float(x), "long" :float(y), "date": date.today(), "radius" : 10}, ignore_index=True)
        
cities_to_point
map_name_to_coor = {r[1][0] : (r[1][1], r[1][2])for r in cities_to_point.iterrows()}
print(map_name_to_coor)

In [ ]:
links_df =[{"name": k, "weight": v, "coordinates" :[map_name_to_coor[c] for c in k]} for k,v in count_dict.items()]


from geojson import LineString, Feature, FeatureCollection
coll = FeatureCollection([Feature(geometry=LineString(l["coordinates"]), properties={"weight": l["weight"]}) for l in links_df])



In [ ]:
from mapboxgl.viz import *
from mapboxgl.utils import *
token = (open(".mapbox_access_token").read().strip())
points = df_to_geojson(cities_to_point, lat='lat', lon='long', precision=3)

print(points)

color_breaks = [0,9,100,1000,10000]
color_stops = create_color_stops(color_breaks, colors='YlGnBu')
# Create the viz from the dataframe
viz = CircleViz(points,
                access_token=token,
                stroke_color='black',
                stroke_width=0.5,
                radius=4,
                center=(8, 47),
                zoom=7,
                style="mapbox://styles/harne/ck50szq0t0hpl1cmx9hxbfz7h")
viz.show()

In [ ]:
print(coll)
viz = LinestringViz(coll,
            access_token=token,
            center=(8, 47),
            zoom=7,
            line_width_property='weight',
            line_width_stops=create_numeric_stops([0,1,5, 50, 100, 200,500,1000,  1500], 0, 20),   
            color_property='weight',
            color_stops=create_color_stops([0,1,5,  50, 100, 200,500,1000,  1500], colors='Blues'),
            opacity = 0.8,
            style="mapbox://styles/harne/ck50szq0t0hpl1cmx9hxbfz7h",
            box_zoom_on = False,
            )
viz.show()

In [ ]:
viz.__dict__

In [ ]:
{k: v for k, v in sorted(count_cities.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
map_name_to_coor
links_df

In [ ]:
svgio = BytesIO()
side = 500
import IPython.display

from io import BytesIO
import numpy as np
import cairo

x_bar = cities_to_point["long"].mean()
y_bar = cities_to_point["lat"].mean()

transform_X = lambda x : (x-x_bar)*120+side/2 
transform_Y = lambda y : 60 - (v[1]-y_bar)*160+side/2


palette = [
    [64/255, 64/255, 122/255,1.0],
    [255/255, 121/255, 63/255,1.0],
    [255/255, 177/255, 66/255,1.0],
    [52/255, 172/255, 224/255,1.0],
    
]
with cairo.SVGSurface(svgio, side, side) as surface:
    context = cairo.Context(surface)

    for k,v in map_name_to_coor.items():
        X, Y = transform_X(v[0]), transform_Y(v[1])
        context.arc(X, Y, 6, 0, 2*np.pi)
        context.set_source_rgba(*palette[0])
        context.fill()
        
    context.set_source_rgba(0,0,0,1)
    
    for k,v in map_name_to_coor.items():
        X, Y = transform_X(v[0]), transform_Y(v[1])
        context.move_to(X-20,Y-10)
        context.show_text(k)
        context.stroke()
        
#for link in links_df:
   #     coord, weight = link["coordinates"], link["weight"]
        #X1,Y1,X2,Y2 = transform_X(coord[0][0]), transform_Y(coord[0][1]), transform_X(coord[1][0]), transform_Y(coord[1][1])
        #context.move_to(X1, Y1)
        #context.line_to(X2, Y2) 
        #print(X1,X2,Y1,Y2)
        #context.set_line_width(weight/20)
        #context.stroke()

IPython.display.SVG(data=svgio.getvalue())

In [ ]:
cities_to_point["long"].mean()

In [ ]:
map_name_to_coor

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10,15)) 

G = nx.Graph()
G.add_nodes_from(map_name_to_coor.keys())
edges = [(l["name"], l["weight"]) for l in  links_df]
for l in links_df:
    G.add_edge(*l["name"], weight=l["weight"])
nx.draw(G, map_name_to_coor, with_labels=True, ax=axes[0])


weight_matrix = {}

for l in links_df:
    weight_matrix[l["name"][0]] = weight_matrix.get(l["name"][0], {})
    weight_matrix[l["name"][1]] = weight_matrix.get(l["name"][1], {})
    weight_matrix[l["name"][0]][l["name"][1]] = l["weight"]
    weight_matrix[l["name"][1]][l["name"][0]] = l["weight"]
    
dist_matrix = {}

for nr in G:
    dist_matrix[nr] = {}
    weight_matrix[nr] = weight_matrix.get(nr,{})
    for nc in G:
        dist_matrix[nr][nc] = 1/weight_matrix[nr].get(nc, 17) *1000

#print(dist_matrix)

from networkx.drawing.layout import kamada_kawai_layout
np.random.seed(seed=400)

layout = kamada_kawai_layout(
    G,
    scale=100,
    dist=dist_matrix,
    center=[10,0], 
    pos=map_name_to_coor)

nx.draw(G,layout,with_labels=True, ax=axes[1])
plt.show()

fig.savefig('../../latex/thesis_template/figures/CFF-NewDistances.pdf')